#### Import Libraries

In [286]:
import pandas as pd
import re

#### Import Datasets

In [318]:
df_ratings = pd.read_csv('stocks_us/ratings-history.csv')
df_is_stocks = pd.read_csv('stocks_us/us_stocks.csv')

In [319]:
rating_set = {'A', 'A+', 'A-', 'AA', 'AA-', 'AAA', 
              'B', 'B+', 'B-', 'BB', 'BB+', 'BB-', 'BBB', 'BBB+', 'BBB-',
              'CC', 'CCC', 'CCC+', 'CCC-', 'D'}

In [320]:
df_ratings = df_ratings[df_ratings['Rating'].isin(rating_set)]

In [321]:
df_ratings = df_ratings.drop_duplicates(subset = df_ratings.columns)

In [322]:
df_ratings['Rating'] = df_ratings['Rating'].str.replace(r'+', '').str.replace(r'-', '')

## Clean Company Names

In [324]:
df_is_stocks['Clean_Name'] = df_is_stocks['Name']
df_ratings['Clean_Name'] = df_ratings['Name']

In [325]:
def clean(company_name):
    
    name = company_name.lower()
    name = re.sub(r'^a-z', '', name)
    name = re.sub(r'inc', '', name)
    name = re.sub(r'ltd', '', name)
    name = re.sub(r'limited', '', name)
    name = re.sub(r'ltd', '', name)
    name = re.sub(r'corporation', '', name)
    name = re.sub(r'holdings', '', name)
    name = re.sub(r'LLC', '', name)   
    name = re.sub(r'\.', '', name)
    name = re.sub(r',', '', name)
    name.strip()
    return name

In [326]:
df_is_stocks['Clean_Name'] = df_is_stocks['Clean_Name'].apply(clean)
df_ratings['Clean_Name'] = df_ratings['Clean_Name'].apply(clean)

In [327]:
df_ratings = pd.merge(df_ratings, df_is_stocks ,on='Clean_Name')

In [328]:
df_ratings = df_ratings.rename(columns = {'Name_x': 'Name'}).drop('Name_y', axis = 1)

In [329]:
df_ratings.groupby('Rating').count()

,Rating Agency Name,Name,Rating Outlook,date,Clean_Name,Symbol,Sector,industry
Rating,,,,,,,,
A,1190,1190,866,1190,1190,1190,965,965
AA,173,173,124,173,173,173,151,151
AAA,20,20,18,20,20,20,20,20
B,1105,1105,946,1105,1105,1105,1021,1021
BB,1909,1909,1690,1909,1909,1909,1694,1694
BBB,2322,2322,1728,2322,2322,2322,2006,2006
CC,16,16,11,16,16,16,15,15
CCC,138,138,118,138,138,138,129,129
D,13,13,9,13,13,13,9,9


## API Function get Data

In [330]:
#!/usr/bin/env python

try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import json

def get_jsonparsed_data(url):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    
    response = urlopen(url)
    data = response.read().decode("utf-8")
    
    return json.loads(data)



## Get Financial Data

In [331]:
Symbol_list = list(set(df_ratings.Symbol))

In [215]:
company_ratios = pd.DataFrame.from_dict(get_jsonparsed_data("https://financialmodelingprep.com/api/v3/ratios/" + Symbol_list[0] + 
                                                         "?apikey=c283e7eb3ddc10dcc0ed8146046dff97"))

company_ratios = company_ratios[0:0]

In [218]:
for Symbol in Symbol_list:
    
    if Symbol.isalpha() == True:
    
        ratios = pd.DataFrame.from_dict(get_jsonparsed_data("https://financialmodelingprep.com/api/v3/ratios/" + 
                                                             Symbol + "?apikey=c283e7eb3ddc10dcc0ed8146046dff97"))
    
        frames = [company_ratios, ratios]
    
        company_ratios = pd.concat(frames)   

In [332]:
company_ratios.head()

,Symbol,date,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,...,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,Year
0,DNR,2019-12-31,0.495854,0.416899,0.00141664,39.910329,0,39.9103,140.603,-100.693,...,2.11802,0.929941,0.929941,0.929941,3.01608,0.360444,NaN,4.92373,0.325382,2019
1,DNR,2018-12-31,0.812225,0.705682,0.105704,31.156244,0,31.1562,147.857,-116.701,...,2.82784,1.7228,1.7228,1.7228,3.7899,0.619248,NaN,5.75903,0.799227,2018
2,DNR,2017-12-31,0.44155,0.319593,0.000126622,47.276130,NaN,NaN,NaN,NaN,...,5.8944,3.59988,3.59988,3.59988,7.14069,0.851208,0.000285957,15.537,1.4837,2017
3,DNR,2016-12-31,0.417826,0.29191,0.00370476,46.742340,NaN,NaN,NaN,NaN,...,-1.29065,5.74714,5.74714,5.74714,-1.49527,1.29142,0.000385743,-6.41776,2.68953,2016
4,DNR,2015-12-31,0.924113,0.659681,0.00753857,29.144331,NaN,NaN,NaN,NaN,...,-0.124077,0.62956,0.62956,0.62956,-0.134113,0.432688,0.120239,-0.665779,0.435684,2015


## Merge Datasets

#### select dates

In [333]:
df_ratings['date'] = df_ratings['date'].astype('datetime64[ns]')
df_ratings['month_rating'] = df_ratings['date'].dt.month
df_ratings['year_change'] = [1 if x < 7 else 0 for x in df_ratings['month_rating']]
df_ratings['year'] = df_ratings['date'].dt.year

In [334]:
df_ratings['Year'] = df_ratings['year'] - df_ratings['year_change']

In [335]:
company_ratios = company_ratios.rename(columns = {'symbol':'Symbol'})

In [336]:
company_ratios['date'] = company_ratios['date'].astype('datetime64[ns]')
company_ratios['Year'] = company_ratios['date'].dt.year

In [337]:
df_ratings = pd.merge(df_ratings, company_ratios, on=['Symbol', 'Year']) 

In [338]:
df_ratings.groupby('Rating').count()

,Rating Agency Name,Name,Rating Outlook,date_x,Clean_Name,Symbol,Sector,industry,month_rating,year_change,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
Rating,,,,,,,,,,,,,,,,,,,,,
A,854,854,625,854,854,854,854,854,854,854,...,688,682,810,697,697,681,688,651,800,646
AA,142,142,100,142,142,142,142,142,142,142,...,118,112,140,117,117,112,118,103,141,94
AAA,20,20,18,20,20,20,20,20,20,20,...,17,17,19,16,16,17,17,16,20,17
B,845,845,734,845,845,845,837,837,845,845,...,738,725,833,716,716,719,738,314,839,705
BB,1601,1601,1428,1601,1601,1601,1565,1565,1601,1601,...,1431,1414,1566,1404,1404,1397,1431,895,1586,1349
BBB,1871,1871,1439,1871,1871,1871,1853,1853,1871,1871,...,1647,1624,1824,1641,1641,1612,1647,1410,1809,1565
CC,12,12,7,12,12,12,12,12,12,12,...,8,9,12,9,9,9,8,1,12,9
CCC,96,96,82,96,96,96,96,96,96,96,...,84,80,92,82,82,80,84,30,94,83
D,7,7,5,7,7,7,7,7,7,7,...,6,5,6,6,6,5,6,0,6,6


In [341]:
df_ratings.to_csv (r'C:stocks_us/rating_dataframe.csv', index = False, header=True)